# Experiment parameters

In [ ]:
import numpy as np
import scipy as sp

from src.network.network import Network
from src.network.config_manager import NetworkConfigManager

In [ ]:
datasets = ["fourclass"] #[a7a", "a9a", "w7a", "w8a", "phishing"]
# max number of iterations for finding saddle with extragradient
num_iter_solution = 500000

# max time for finding saddle with extragradient
max_time_solution = 3600
# tolerance of solution obtained with extragradient
tolerance_solution = 1e-10
# required accuracy of sliding
eps = 1e-10
seed = 30

In [ ]:
# number of nodes
num_nodes = 25

regcoef_x = 2.0
regcoef_y = 2.0
r_x = 5.0
r_y = 0.0
comm_budget_experiment = 5000

In [ ]:
topology = "ring"

In [ ]:
network = Network(
    comm_budget_experiment,
    num_nodes,
    "mix_mat",
    NetworkConfigManager(f"src/experiment/configs/{topology}.yaml"),
    "src/experiment/configs/network",
)

# Dataset

In [ ]:
from libsvmdata import fetch_libsvm

In [ ]:
A, b = fetch_libsvm(datasets[0])
if sp.sparse.issparse(A):
    A = A.toarray()

In [ ]:
A.shape

In [ ]:
from src.oracles.base import ArrayPair

In [ ]:
z_0 = ArrayPair.zeros(A.shape[1])

# Oracles

In [ ]:
from src.utils.utils import get_oracles

In [ ]:
oracles, oracle_mean, L, delta, mu, A_grad, b_grad = get_oracles(
    A,
    b,
    num_nodes,
    regcoef_x,
    regcoef_y,
    r_x,
    r_y,
)

# Run

## Solving linear regression

In [ ]:
# x = np.linalg.inv(A_theor) @ b_theor
# x = np.linalg.solve(A_grad, b_grad)

In [ ]:
x = np.linalg.lstsq(A_grad, b_grad, rcond=None)[0]

In [ ]:
np.linalg.norm(x)

In [ ]:
z_true = ArrayPair(x, np.zeros(A.shape[1]))

In [ ]:
z_true.x

In [ ]:
g_true = ArrayPair(
    np.zeros((num_nodes, z_true.x.shape[0])), np.zeros((num_nodes, z_true.y.shape[0]))
)

## Centralized Extragradient

In [ ]:
from src.utils.utils import solve_with_extragradient_real_data

In [ ]:
z_true = solve_with_extragradient_real_data(
    A=A,
    b=b,
    regcoef_x=regcoef_x,
    regcoef_y=regcoef_y,
    r_x=r_x,
    r_y=r_y,
    num_iter=num_iter_solution,
    max_time=max_time_solution,
    tolerance=tolerance_solution,
)

In [ ]:
z_true.x

## Decentralized Extragradient method

In [ ]:
from src.experiment.decentralized_extragradient_gt import run_extragrad_gt

In [ ]:
extragrad = run_extragrad_gt(
    oracles=oracles,
    L=L,
    mu=mu,
    z_0=z_0,
    z_true=z_true,
    g_true=g_true,
    network=network,
    r_x=r_x,
    r_y=r_y,
    comm_budget_experiment=comm_budget_experiment,
)

## Decentralized Extragradient method with consensus subroutine

In [ ]:
from src.experiment.decentralized_extragradient_con import run_extragrad_con

In [ ]:
network.current_state = 0

In [ ]:
extragrad_con = run_extragrad_con(
    oracles=oracles,
    L=L,
    mu=mu,
    z_0=z_0,
    z_true=z_true,
    g_true=g_true,
    network=network,
    r_x=r_x,
    r_y=r_y,
    eps=eps,
    comm_budget_experiment=comm_budget_experiment,
)

## Decentralized gradient sliding

In [ ]:
from src.experiment.saddle_sliding import run_sliding

In [ ]:
network.current_state = 0

In [ ]:
sliding = run_sliding(
    oracles=oracles,
    L=L,
    delta=delta,
    mu=mu,
    z_0=z_0,
    z_true=z_true,
    g_true=g_true,
    network=network,
    r_x=r_x,
    r_y=r_y,
    eps=eps,
    comm_budget_experiment=comm_budget_experiment,
)

## Decentralized Algorithm 1

In [ ]:
from src.experiment.decentralized_vi_papc import run_vi_papc

In [ ]:
network.current_state = 0
network.matrix_type = "gos_mat"

In [ ]:
vi_papc = run_vi_papc(
    num_nodes=num_nodes,
    oracles=oracles,
    L=L,
    mu=mu,
    z_0=z_0,
    z_true=z_true,
    g_true=g_true,
    network=network,
    r_x=r_x,
    r_y=r_y,
    comm_budget_experiment=comm_budget_experiment,
)

## Decentralized Alogorithm 2

In [ ]:
from src.experiment.decentralized_vi_adom import run_vi_adom

In [ ]:
network.current_state = 0
network.matrix_type = "gos_mat"

In [ ]:
batch_size = len(oracles)
L_avg = L

In [ ]:
x_0 = ArrayPair.zeros(A.shape[1])
y_0 = ArrayPair.zeros(A.shape[1])        

In [ ]:
vi_adom = run_vi_adom(
    num_nodes=num_nodes,
    oracles=oracles,
    b=batch_size,
    L=L,
    L_avg=L_avg,
    mu=mu,
    x_0=x_0,
    y_0=y_0,
    z_0=z_0,
    z_true=z_true,
    g_true=g_true,
    network=network,
    r_x=r_x,
    r_y=r_y,
    comm_budget_experiment=comm_budget_experiment,
)

# Preplot

In [ ]:
from src.experiment.plotting import preplot_algorithms

In [ ]:
#labels = ['EGD-GT', 'EGD-CON', 'Sliding', 'Alg.1']
labels = ['EGD-GT', 'EGD-CON', 'Sliding', 'Alg.2']

In [ ]:
preplot_algorithms(
    topology="cycle",
    num_nodes=num_nodes,
    data=datasets[0],
    labels=labels,
    #runners=[extragrad, extragrad_con, sliding, vi_papc],
    runners=[extragrad, extragrad_con, sliding, vi_adom],
    dist_to_opt_type="argument",
)

# Save

In [ ]:
from src.experiment.saving import save_algorithms

In [ ]:
#method_names = ['extragrad', 'extragrad_con', 'sliding', 'vi_papc']
method_names = ['extragrad', 'extragrad_con', 'sliding', 'vi_adom']

In [ ]:
save_algorithms(
    topology="cycle",
    num_nodes=num_nodes,
    data=datasets[0],
    #methods=[extragrad, extragrad_con, sliding, vi_papc],
    runners=[extragrad, extragrad_con, sliding, vi_adom],
    method_names=[f"{method_name}.pkl" for method_name in method_names],
    z_true=z_true,
    experiment_type="real",
)

# Plot

In [ ]:
from src.experiment.plotting import plot_algorithms

In [ ]:
plot_algorithms(
    topology="cycle",
    num_nodes=num_nodes,
    data=datasets[0],
    labels=labels,
    method_names=method_names,
)